In [ ]:
import mbuild as mb
import foyer
import parmed
import subprocess
import unyt as u
from mbuild.lib.recipes.alkane import Alkane
from mbuild.formats import par_writer

# Building n-butane
cmpd=Alkane(4)
cmpd.name='C4A'
mb.compound.Compound.energy_minimize(cmpd)
mb.compound.Compound(name='C4A')
cmpd.visualize()


In [ ]:
# Build initial configuratoins at the experimental density 
vapor_density = 0.0233 * u.g/u.cm**3 # cm**3 = mL, but unyt doesn't support liter quantities
liquid_density = 0.505 * u.g/u.cm**3

#default units for mb.fill_box are kg/m^3; hence the need to convert units
vapor_density.convert_to_units(u.kg/u.m**3) # convert to kg/m**3
liquid_density.convert_to_units(u.kg/u.m**3)

#Using 50 molecules in the gas phase 
vapor_box = mb.fill_box(cmpd, n_compounds=50, density=vapor_density)
liquid_box = mb.fill_box(cmpd, n_compounds=250, density=liquid_density)



In [ ]:
vapor_box.visualize()

In [ ]:
liquid_box.visualize()

In [ ]:
#apply force field
ff=foyer.Forcefield(forcefield_files='opls_gomc.xml')

#residues=cmpd.name is used to set the residue name in the PDB and PSF
#need to include assert_dihedral=False to get foyer to apply parameters
#verbose=True will cause an error, hence it is set to False
vapor_box_param=ff.apply(vapor_box, residues=cmpd.name, assert_dihedral_params=False, verbose=False)
liquid_box_param=ff.apply(liquid_box, residues=cmpd.name,assert_dihedral_params=False, verbose=False)

#write CHARMM-style parameter file
#Versions of GOMC >=2.5 will read this without modification
parameter_file='mbuild_opls.par'
par_writer.write_par(vapor_box_param,parameter_file)


In [ ]:
# Save PDB and PSF files
vapor_box_param.save('coords0.pdb', overwrite=True, use_hetatoms=False)
vapor_box_param.save('structure0.psf', overwrite=True)
liquid_box_param.save('coords1.pdb', overwrite=True, use_hetatoms=False)
liquid_box_param.save('structure1.psf', overwrite=True)

#obtain box length: 0=x; 1=y; 2=z
box_len_vap=[]
box_len_liq=[]
for i in range(0,3):
    box_len_vap.append(vapor_box_param.box[i])
for i in range(0,3):  
    box_len_liq.append(liquid_box_param.box[i])


In [ ]:
def write_gomc_gemc_input(filename, box_len_vap, box_len_liq, 
                          temperature, run_steps, eq_steps, parameters,
                          output, console, console_freq,
                          coords, coord_freq, 
                          restart, restart_freq,
                          block_ave, block_ave_freq,
                          coords0, structure0,
                          coords1, structure1):
   
    """ Write GOMC input file for a GEMC simulation 
    
    Some simulation input parameters have been hard-coded in accordance with the 
    TraPPE specification. 
    MC moves, CBMC parameters, and output control have been adapted from
    https://github.com/GOMC-WSU/GOMC_Examples/tree/master/NVT_GEMC/pure_fluid/octane_T_360_00_K
    Gibbs Ensemble Monte Carlo involves simultaneous simulations of 
    two boxes (generally corresponding to two different phases)
    
    Parameters
    ----------
    filename : str
    param_structure0: parmed.Structure
        Should be parametrized
    param_structure1: parmed.Structure
        Should be parametrized
    temperature : float
        Implicitly in Kelvin
    parameters : str
        CHARMM/NAMD-style PAR file for force field information
    coords0 : str
        PDB file for param_structure0 coordinates and atom names
    structure0: str
        PSF file for param_structure0 atomtypes, bonding information, and topology
    coords1 : str
        PDB file for param_structure0 coordinates and atom names
    structure1: str
        PSF file for param_structure0 atomtypes, bonding information, and topology    
    output : str
        Prefix for output files from simulation
    runsteps : float
        Number of Monte Carlo Steps to run the simulation
        
    Notes
    -----
    Box dimensions are assumed to be orthorhombic
    GOMC input files utilize Angstroms, which are also the units of the parmed Structure box
    TraPPE uses 14 Angstrom cutoffs, but here we use 10 Angstrom because our simulations are small
    """
    with open(filename, 'w') as f:
        f.write(""" 
###########################################################################
#  ========-------------------- INPUT --------------------------===========
############################################################################

#########################
# restart from prior simulation
#########################
Restart     false


####################################
# kind (RESTART, RANDOM, INTSEED)
####################################
PRNG        RANDOM

####################################
# FORCE FIELD
####################################
ParaTypeCHARMM   true
Parameters      {parameters}

####################################
# INPUT PDB FILES
####################################
Coordinates 0    {coords0}
Coordinates 1    {coords1}

####################################
# INPUT PSF FILES
####################################
Structure 0      {structure0}
Structure 1      {structure1}

############################################################################
#  =======--------------------- SYSTEM --------------------------===========
############################################################################

##################################
# GEMC TYPE (DEFULT IS NVT_GEMC)
##################################
GEMC      NVT

#############################
# SIMULATION CONDITIONS
#############################
Temperature     {temperature}
Potential       VDW
LRC         true
Rcut        10
Exclude     1-4
RcutCoulomb  0  14.0
RcutCoulomb  1  14.0

#############################
# ELECTROSTATICS
#############################
ElectroStatic   true
Ewald           true
CachedFourier   true
Tolerance      0.00001
1-4scaling     1.0

################################
# PRESSURE CALCULATION FREQ
################################
PressureCalc  true  1000

################################
# STEPS
################################
RunSteps       {run_steps}
EqSteps        {eq_steps}
AdjSteps       1000

################################
# MOVE FREQUENCY
################################
DisFreq           0.49
RotFreq           0.10
VolFreq           0.01
SwapFreq          0.20
RegrowthFreq      0.10
CrankShaftFreq    0.10

################################
# BOX DIMENSION #, X, Y, Z
################################
CellBasisVector1 0  {box0_x}  0.00    0.00
CellBasisVector2 0  0.00    {box0_y}  0.00
CellBasisVector3 0  0.00    0.00    {box0_z}

CellBasisVector1 1  {box1_x}  0.00    0.00
CellBasisVector2 1  0.00    {box1_y}  0.00
CellBasisVector3 1  0.00    0.00    {box1_z}

##############################
# CBMC TRIALS
##############################
CBMC_First   10
CBMC_Nth     8
CBMC_Ang     50
CBMC_Dih     50

############################################################################
#  =======-------------------- OUTPUT --------------------------===========
############################################################################

##########################
# Output file
##########################
OutputName  {output}

#####################################
# enable, frequency
#####################################
CoordinatesFreq    {coords}   {coord_freq}
RestartFreq        {restart}  {restart_freq}
ConsoleFreq        {console}  {console_freq}
BlockAverageFreq   {block_ave} {block_ave_freq}

##################################
# enable: blk avg., fluct.
##################################
OutEnergy         true    true
OutPressure       true    true
OutMolNum         true    true
OutDensity        true    true
""".format(temperature=temperature, parameters=parameters,
           run_steps=run_steps, eq_steps=eq_steps,
           console=console, console_freq=console_freq,
           coords=coords, coord_freq=coord_freq,
           restart=restart, restart_freq=restart_freq,
           block_ave=block_ave, block_ave_freq=block_ave_freq,
           structure0=structure0, coords0=coords0, 
           structure1=structure1, coords1=coords1,
           box0_x=box_len_vap[0], box0_y=box_len_vap[1], box0_z=box_len_vap[2],
           box1_x=box_len_liq[0], box1_y=box_len_liq[1], box1_z=box_len_liq[2],
           output=output))
        
        
       

In [ ]:
# define run parameters
temperature = 350 * u.Kelvin
run_steps=10000000
eq_steps=1000000
input_file='in.conf'
coords='true'
coord_freq=1000000
restart='true'
restart_freq=1000000
console='true'
console_freq=10000
block_ave='true'
block_ave_freq=1000000
output='C4A_out'

write_gomc_gemc_input(input_file, box_len_vap, box_len_liq,
                      temperature.value,  run_steps, eq_steps, parameter_file, 
                      output, console, console_freq, 
                      coords, coord_freq, restart, restart_freq,
                      block_ave, block_ave_freq,
                      coords0='coords0.pdb', structure0='structure0.psf',
                      coords1='coords1.pdb', structure1='structure1.psf') 
    

In [ ]:
# this is a slow simulation, and it's recommended to run it on something that isn't a 
# laptop.  But just in case you're at Vanderbilt in March and need to warm your hands up,
# you can try this. "+p2" tells GOMC to run on 2 CPUs.
p = subprocess.Popen('GOMC_CPU_GEMC +p2 in.conf', shell=True, 
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                    universal_newlines=True)
out, err = p.communicate()
with open("gomc.out", 'w') as f:
    f.write(out)
with open("gomc.err", 'w') as f:
    f.write(err)